In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
import joblib
from sklearn import metrics
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
df_data = pd.read_csv("feature_dataset/Dataset_an.csv")
df_unkonw = pd.read_csv("feature_dataset/Dataset_unan.csv")

In [3]:
feat = []
for i in df_data.columns:
    if i != "GeneID":
        if i != "ps":
            feat.append(i)
def lgb_p(x):
    y = 0
    if x > 0.5:
        y = 1
    return y

In [4]:
df11 = pd.DataFrame()

rf = RandomForestClassifier(n_estimators=100, random_state=2020)

clf = rf.fit(df_data[feat],df_data["ps"])
rf_y = clf.predict(df_unkonw[feat])

model=CatBoostClassifier(
            loss_function="Logloss",
            eval_metric="AUC",
            task_type="CPU",
            learning_rate=0.1,
            iterations=2,
            random_seed=2020,
            od_type="Iter",
            depth=8,
            early_stopping_rounds=500)

clf = model.fit(df_data[feat],df_data["ps"],verbose=500)
cat_y = clf.predict(df_unkonw[feat])

model = XGBClassifier(max_depth=8, silent=0,booster='gbtree',
                      random_state=2020)
model.fit(df_data[feat],df_data["ps"],eval_metric="auc")
xgb_y = model.predict(df_unkonw[feat])

train_lgb = lgb.Dataset(data=df_data[feat],label=df_data["ps"])
param = {'num_leaves':8, 'num_trees':8, 'objective':'binary','boost_from_average': True,'force_row_wise':True,'metric':'auc','data_random_seed': 2020,'seed': 2020}
bst = lgb.train(param, train_lgb)
y_pred = bst.predict(df_unkonw[feat])
df11["lgb"] = y_pred
lgb_y = df11["lgb"].apply(lambda x:lgb_p(x))

def vote(x):
        y = 0
        if abs(x) > 2:
            y = 1
        return y

df_cat = pd.DataFrame(cat_y)#2分
df_xgb = pd.DataFrame(xgb_y)#1分
df_rf = pd.DataFrame(rf_y)#1.5分
df_lgb = pd.DataFrame(list(lgb_y))
df = pd.DataFrame(df_lgb[0]*1 + df_xgb[0]*2 + df_rf[0]*1 + df_cat[0]*1)
df[1] = df[0].apply(lambda x : vote(x))

0:	total: 76.1ms	remaining: 76.1ms
1:	total: 105ms	remaining: 0us
[22:54:16] WARNING: ../src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[LightGBM] [Info] Number of positive: 220, number of negative: 405
[LightGBM] [Info] Total Bins 68644
[LightGBM] [Info] Number of data points in the train set: 625, number of used features: 331
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.352000 -> initscore=-0.610260
[LightGBM] [Info] Start training from score -0.610260


In [5]:
df.index = df_unkonw["GeneID"]

In [6]:
df = df[df[1] == 1]

In [7]:
del df[0]
del df[1]

In [8]:
df.to_csv('supplement/predict_new_photosynthesis_related_gene.csv')

In [9]:
len(df)

716